## Before Beginning

**Note that for notebooks** the multiprocessing start method should be set to `fork` (default on Linux).
To use with `spawn` (default on Windows and macOS), use the `multiprocess` library.

Lets check libEnsemble is installed.

In [ ]:
!pip install libensemble

## Random sample with simple function

libEnsemble initializes a *manager* process and as many *workers*  as the user requests.

For this example, our generator will produce 2D uniform randomly sampled values, and our simulator will calculate the six_hump_camel function for each. 

In [ ]:
import numpy as np
import pprint

from libensemble import Ensemble
from libensemble.gen_funcs.sampling import uniform_random_sample
from libensemble.sim_funcs.six_hump_camel import six_hump_camel
from libensemble.specs import ExitCriteria, GenSpecs, LibeSpecs, SimSpecs

if __name__ == "__main__":

    nworkers = 4
    libE_specs = LibeSpecs(nworkers=nworkers)

    sim_specs = SimSpecs(
        sim_f=six_hump_camel,
        inputs=["x"],
        outputs=[("f", float)],
    )

    gen_specs = GenSpecs(
        gen_f=uniform_random_sample,
        outputs=[("x", float, 2)],
        user={
            "gen_batch_size": 50,
            "lb": np.array([-3, -2]),
            "ub": np.array([3, 2]),
        },
    )

    exit_criteria = ExitCriteria(sim_max=100)

    sampling = Ensemble(
        libE_specs=libE_specs,
        sim_specs=sim_specs,
        gen_specs=gen_specs,
        exit_criteria=exit_criteria,
    )

    sampling.add_random_streams()
    H, persis_info, flag = sampling.run()

    # Print first 10 lines of input/output values
    if sampling.is_manager:
        pprint.pp(H[["x", "f"]][:10])
        sampling.save_output("simple")

Note that the `sampling.is_manager` is not required here. However, this script could also be run via the mpirun interface. In that case, it would be necessary to ensure only the manager process outputs results.